## How to Manage Secrets

https://blog.cryptomove.com/secrets-management-guide-approaches-open-source-tools-commercial-products-challenges-db560fd0584d

### Why

__DevOps processes & microservices based architecture__

leads to secrets proliferation. Teams undergoing DevOps transformations move fast and manage many different infrastructure environments and services for development, testing, integration, and deployment. Secrets management for DevOps environments is vital as part of the secure software development lifecycle.

### What

__Type of secrets__
* AWS access keys
* Database credentials (host, port, username, password)
* API tokens
* Pem, Cert files
* Accounts: url, username, password, email, secure Q&A

### Who

__Top providers__

https://stackshare.io/secrets-management

* [Vault](https://learn.hashicorp.com/vault/) (Hashicorp)
* AWS [Key Management Service (KMS)](https://aws.amazon.com/kms/) 
* [Knox](https://github.com/pinterest/knox) (Pinterest)
* [Confidant](https://github.com/lyft/confidant) (Lyft)
* [Docker secrets](https://blog.docker.com/2017/02/docker-secrets-management/)
* [Keywhiz](https://square.github.io/keywhiz/) (Square)

### How

#### Vault by Hashicorp

* [Review](https://thenewstack.io/using-vault-to-manage-your-apps-secrets/)
* [Blog](https://www.prodops.io/blog/taking-your-hashicorp-vault-to-the-next-level)
* [Get started](https://learn.hashicorp.com/vault/)

##### Install

* download one binary

* add to PATH, set 2 env vars

```
$ export VAULT_ADDR="http://127.0.0.1:8200"
$ export VAULT_DEV_ROOT_TOKEN_ID="s.4sSqAnf111111xxxxxxxxxx"

```

* ready to go

```
$ vault
$ vault server -dev    # start dev server
$ vault status
```

##### Get started - CLI

###### write a secret

```
$ vault kv put secret/hello foo=world
$ vault kv put secret/test cloud_provider=aws today=2019-02-09

```

###### get secret

```
$ vault kv get secret/hello
$ vault kv get -field=today secret/hello
$ vault kv get -format=json secret/hello
$ vault kv get -format=json secret/hello | jq -r .data.data.today
```

######  delete 

```
$ vault kv delete secret/test
```

###### enable another secrets engine besides default secret

```
$ vault secrets enable kv
$ vault write kv/my-secret value="s3c(eT"
$ vault write kv/hello target=world
$ vault write kv/airplane type=boeing class=787
$ vault list kv
```

##### [Python API](https://github.com/hvac/hvac)

```
$ pip install hvac
```

In [1]:
import os, hvac

In [2]:
client = hvac.Client(url=os.environ['VAULT_ADDR'], token=os.environ['VAULT_DEV_ROOT_TOKEN_ID'])

In [3]:
print(client.read('kv/hello'))

{'request_id': '7ca92b81-7711-01f6-6d62-af123ad92e50', 'lease_id': '', 'renewable': False, 'lease_duration': 2764800, 'data': {'target': 'world'}, 'wrap_info': None, 'warnings': None, 'auth': None}


In [4]:
client.write('kv/postgresql_dev', hostname='metadb.cjnq5tu75jam.us-east-1.rds.amazonaws.com', \
             port='5432', user='wgong', pwd='PostGreSQ121', db_name='traffic_db')

In [5]:
db_secrets = client.read('kv/postgresql_dev')
print(db_secrets)

{'request_id': '665174ed-73d9-4ea7-a11e-41510deef9ec', 'lease_id': '', 'renewable': False, 'lease_duration': 2764800, 'data': {'db_name': 'traffic_db', 'hostname': 'metadb.cjnq5tu75jam.us-east-1.rds.amazonaws.com', 'port': '5432', 'pwd': 'PostGreSQ121', 'user': 'wgong'}, 'wrap_info': None, 'warnings': None, 'auth': None}


In [6]:
db_host,db_port,table_name,db_user,db_pwd = \
    db_secrets['data']['hostname'], \
    db_secrets['data']['port'], \
    db_secrets['data']['db_name'], \
    db_secrets['data']['user'], \
    db_secrets['data']['pwd']

In [7]:
print([db_host,db_port,table_name,db_user,db_pwd])

['metadb.cjnq5tu75jam.us-east-1.rds.amazonaws.com', '5432', 'traffic_db', 'wgong', 'PostGreSQ121']


##### Web UI

* http://127.0.0.1:8200/ui

<img src=vault.png/>